In [2]:
import pandas as pd
import numpy as np

In [3]:
shoes = pd.read_csv('~/Documents/GitHub/DSBA-webscraping/analysis/output.csv')  

for col in shoes:
    shoes[col] = shoes[col].str.strip().str.replace(u'\xa0', ' ')

shoes['regular_price'] = shoes['regular_price'].str.replace(' ', '').astype('float64')
shoes['special_price'] = shoes['special_price'].str.replace(' ', '').astype('float64')
shoes['shoe_type'] = shoes['first_name'].str.split(' ', n=1, expand=True)[0]
shoes['regular_if_discounted'] = shoes['regular_price'].where(shoes['special_price'].notna())

shoes = shoes[['first_name', 'second_name', 'shoe_type', 'regular_price', 'regular_if_discounted', 'special_price']]

shoes[:5]

,first_name,second_name,shoe_type,regular_price,regular_if_discounted,special_price
0,Sandały EVA MINGE,EM-44-07-000811 121,Sandały,399.0,399.0,299.0
1,Sandały NESSI,22179 Biały,Sandały,339.0,NaN,NaN
2,Sandały BALDACCINI,1697000 Chic Złoto,Sandały,349.0,349.0,279.0
3,Sneakersy ROBERTO,3034 Złoto Lico,Sneakersy,449.0,449.0,309.0
4,Klapki EVA MINGE,EM-41-09-001139 101,Klapki,389.0,389.0,289.0


In [26]:
def get_perc(prop, n=2):
    return str(round(prop*100,n)) + '%'

def summarise(df, grp_col):
    df = df.groupby(grp_col).agg(**{
        'Shoes count': ('regular_price', 'count'),
        'Shoes on promo': ('special_price', 'count'),
        'Avg. regular price': ('regular_price', lambda x: round(np.mean(x), 2)),
        'Avg. regular price if discounted': ('regular_if_discounted', lambda x: round(np.mean(x), 2)),
        'Avg. promo price': ('special_price', lambda x: round(np.mean(x), 2))
    })

    df = df.assign(**{
        'as % of total': lambda x: (x['Shoes count'] / x['Shoes count'].sum()).apply(get_perc, n=1),
        'Shoes discounted (%)': lambda x: (x['Shoes on promo'] / x['Shoes count']).apply(get_perc, n=1),
        'Avg. discount (%)': lambda x: (x['Avg. promo price'] / x['Avg. regular price if discounted'] - 1).apply(get_perc, n=1)
    }).sort_values(by='Shoes count', ascending=False)

    cols = ['Shoes count', 'as % of total']
    df = df[cols + [x for x in df.columns if x not in cols]]

    return df

total_summary = summarise(shoes.assign(shoe_type='total'), 'shoe_type')
total_summary

,Shoes count,as % of total,Shoes on promo,Avg. regular price,Avg. regular price if discounted,Avg. promo price,Shoes discounted (%),Avg. discount (%)
shoe_type,,,,,,,,
total,46304,100.0%,36332,448.67,478.65,319.72,78.5%,-33.2%


In [27]:

type_summary = summarise(shoes, 'shoe_type')
type_summary.head(10)

,Shoes count,as % of total,Shoes on promo,Avg. regular price,Avg. regular price if discounted,Avg. promo price,Shoes discounted (%),Avg. discount (%)
shoe_type,,,,,,,,
Sandały,8561,18.5%,5891,374.48,395.60,275.32,68.8%,-30.4%
Sneakersy,7681,16.6%,6203,492.71,502.87,337.68,80.8%,-32.8%
Botki,6390,13.8%,5863,596.99,620.54,383.85,91.8%,-38.1%
Klapki,5059,10.9%,3494,345.17,378.76,259.72,69.1%,-31.4%
Półbuty,2998,6.5%,2237,378.90,399.63,282.11,74.6%,-29.4%
Buty,2700,5.8%,2260,484.13,482.70,350.72,83.7%,-27.3%
Espadryle,2323,5.0%,1663,360.19,414.18,278.74,71.6%,-32.7%
Japonki,1295,2.8%,987,228.82,229.32,154.91,76.2%,-32.4%
Trapery,1210,2.6%,1104,659.39,682.29,425.54,91.2%,-37.6%


In [28]:
pd.concat([total_summary, type_summary])

,Shoes count,as % of total,Shoes on promo,Avg. regular price,Avg. regular price if discounted,Avg. promo price,Shoes discounted (%),Avg. discount (%)
shoe_type,,,,,,,,
total,46304,100.0%,36332,448.67,478.65,319.72,78.5%,-33.2%
Sandały,8561,18.5%,5891,374.48,395.60,275.32,68.8%,-30.4%
Sneakersy,7681,16.6%,6203,492.71,502.87,337.68,80.8%,-32.8%
Botki,6390,13.8%,5863,596.99,620.54,383.85,91.8%,-38.1%
Klapki,5059,10.9%,3494,345.17,378.76,259.72,69.1%,-31.4%
Półbuty,2998,6.5%,2237,378.90,399.63,282.11,74.6%,-29.4%
Buty,2700,5.8%,2260,484.13,482.70,350.72,83.7%,-27.3%
Espadryle,2323,5.0%,1663,360.19,414.18,278.74,71.6%,-32.7%
Japonki,1295,2.8%,987,228.82,229.32,154.91,76.2%,-32.4%
